## Red neuronal
El código tiene como objetivo entrenar y guardar un modelo de red neuronal para predecir si un punto en el mapa representa un incendio o no. En resumen, el código carga los datos de un archivo CSV, divide los datos en conjuntos de entrenamiento y prueba, normaliza las variables, define la arquitectura de la red neuronal, compila el modelo, entrena la red neuronal y evalúa la precisión de la red. Finalmente, guarda el modelo y los pesos del modelo en dos archivos diferentes. Además, se guarda un escalador para normalizar las variables cuando se hacen predicciones en nuevos datos. También se calcula una matriz de confusión para evaluar la precisión del modelo.

In [ ]:
from pickle import dump
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
# Cargar datos de entrenamiento
dataset = pd.read_csv('./CompleteDataset.csv')
X = dataset.iloc[:, :15].values
y = dataset.iloc[:, 15].values

In [ ]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

In [ ]:
# escalado de variables (normalización)
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
# guardar el escalador para usarlo en la predicción de nuevos datos
dump(sc_X, open('scaler.pkl', 'wb'))

In [ ]:
# Definir la arquitectura de la red
model = keras.Sequential()
model.add(keras.layers.Dense(units=16, kernel_initializer="uniform",
                             activation="sigmoid", input_dim=14))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# Compilar el modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [ ]:
# Entrenar la red
model.fit(X_train, y_train, epochs=50, batch_size=32,
          validation_data=(X_test, y_test))

In [ ]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

In [ ]:
# Precision del modelo
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# convertir a binario para la matriz de confusión
y_train_bin = to_categorical(y_train)
y_test_bin = to_categorical(y_test)
y_pred = model.predict(X_test)
y_pred_bin = to_categorical(y_pred.round())

# matriz de confusión
cm = confusion_matrix(
    y_test_bin.argmax(axis=1), y_pred_bin.argmax(axis=1))
print(cm)


In [ ]:
model.save('model.h5')
model.save_weights('weights.h5')